In [1]:
%load_ext autoreload
%autoreload 2

In [60]:
from pathlib import Path
import os 
import pandas as pd
from sklearn.model_selection import KFold
import numpy as np 
from src.dm import DataModule
from src.module import Module

path = Path('/fastdata/contrails')

In [31]:
train_records = os.listdir(path / 'train')
validation_records = os.listdir(path / 'validation')

len(train_records), len(validation_records)

(20529, 1856)

In [35]:
100 * len(validation_records) / len(train_records) 

9.040869014564763

In [44]:
folds = KFold(n_splits=10, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(folds.split(train_records)):
    fold_train = pd.DataFrame(np.array(train_records)[train_index]).to_csv(path / f'fold_{i}.csv', index=False)
    fold_val = pd.DataFrame(np.array(train_records)[test_index]).to_csv(path / f'fold_{i}_val.csv', index=False)

In [45]:
os.listdir(path)

['fold_7_val.csv',
 'train',
 'sample_submission.csv',
 'fold_3_val.csv',
 'fold_5_val.csv',
 'fold_0_val.csv',
 'stats.csv',
 'fold_0.csv',
 'validation_metadata.json',
 'fold_8_val.csv',
 'fold_9_val.csv',
 'fold_9.csv',
 'fold_1.csv',
 'fold_4.csv',
 'fold_1_val.csv',
 'fold_5.csv',
 'fold_7.csv',
 'fold_3.csv',
 'fold_2.csv',
 'fold_6.csv',
 'fold_2_val.csv',
 'fold_8.csv',
 'fold_0_train.csv',
 'train_metadata.json',
 'validation',
 'fold_6_val.csv',
 'test',
 'fold_4_val.csv']

In [57]:
fold_train_records = pd.read_csv(path / 'fold_0.csv')
# fold_train_records['0'].values

fold_train_records

,0
0,2175739284570639142
1,6624199988481307860
2,3724289921967457485
3,7207816703707783652
4,1814587943104364094
...,...
18471,41326737158326673
18472,8174741996149584540
18473,2788202044994308726
18474,9112310559916158355


In [58]:
fold_val_records = pd.read_csv(path / 'fold_0_val.csv')
fold_val_records

,0
0,1144343458050482940
1,5772265529513313587
2,6509676037330853564
3,4970426288017777995
4,2409525184733690893
...,...
2048,759463280039055451
2049,8275080707001327959
2050,7332692663844231941
2051,3735988636526973931


In [71]:
fold = 1 

dm = DataModule(false_color=True, t=[4], fold=fold)
dm.setup()

In [72]:
# images, masks = next(iter(dm.train_dataloader()))
images, masks = next(iter(dm.val_dataloader()))

images.shape, masks.shape

(torch.Size([16, 256, 256, 1, 3]), torch.Size([16, 256, 256]))

In [75]:
import lightning as L

t = [4]
bands = [8,9,10]
false_color = False

dm = DataModule(false_color=false_color, t=t, bands=bands, batch_size=1, num_workers=0, pin_memory=False)
module = Module(dict(
    encoder='resnet18', 
    pretrained=True, 
    t=len(t), 
    in_chans=len(bands) if not false_color else 3,
    loss='dice',
    optimizer='Adam',
    optimizer_params=dict(lr=1e-3),
))

trainer = L.Trainer(
    accelerator="cuda",
    devices=1,
    precision=16,
    max_epochs=100,
    # limit_train_batches=10,
    # limit_val_batches=10,
    overfit_batches=1
)

trainer.fit(module, dm)

/home/juan/.local/lib/python3.10/site-packages/lightning/fabric/connector.py:555: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type     | Params
------------------------------------
0 | model  | Unet     | 14.3 

Sanity Checking: 0it [00:00, ?it/s]

/home/juan/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/juan/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:261: UserWarning: You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.
  rank_zero_warn(
/home/juan/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataL

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
